# XGBoost Classification Master NB

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import joblib

import warnings
warnings.filterwarnings("ignore")

# ------------------------ #

# 1) Load the dataset 
df = pd.read_excel('sample_data.xlsx') 

# 2) Separate features and target variable from the dataset
X = df.drop(columns=["Target"])
y = df["Target"]

# Separate numeric and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# 3) Create transformers for data preprocessing
# No need for handling missing values since XGBoost can manage them natively.
categorical_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')  # Keep numeric columns as is

# 4) Create a pipeline with an XGBoost Classifier
base_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))  # XGBoost doesn't need label encoding for binary classification
])

# 5) Perform a train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 6) Train and evaluate the Base Model
base_pipeline.fit(X_train, y_train)

# 7) Predictions and evaluation metrics
print("EVAL METRICS FOR BASE MODEL")
y_train_pred = base_pipeline.predict(X_train)
y_test_pred = base_pipeline.predict(X_test)

def evaluate_model(y_true, y_pred, data_type="Train"):
    print(f"Evaluation metrics for {data_type} data:")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print("\n")

evaluate_model(y_train, y_train_pred, "Train")
evaluate_model(y_test, y_test_pred, "Test")

# 8) Hyperparameter Tuning
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__learning_rate': [0.01, 0.1, 0.5, 1.0],
    'classifier__max_depth': [3, 5, 7],  # Typical values for max_depth in XGBoost
}

grid_search = GridSearchCV(base_pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"Best Hyperparameters: {grid_search.best_params_}")

# 9) Build and evaluate the Best Model
print("")
print("----EVAL METRICS FOR BEST MODEL----")
print("")

best_pipeline = grid_search.best_estimator_

y_train_best_pred = best_pipeline.predict(X_train)
y_test_best_pred = best_pipeline.predict(X_test)

evaluate_model(y_train_best_pred, y_train, "Train (Best Model)")
evaluate_model(y_test_best_pred, y_test, "Test (Best Model)")

# Compare metrics
metrics_comparison = pd.DataFrame({
    'Metric': ['Accuracy', 'F1 Score'],
    'BaseModelTrain': [accuracy_score(y_train, y_train_pred), f1_score(y_train, y_train_pred)],
    'BaseModelTest': [accuracy_score(y_test, y_test_pred), f1_score(y_test, y_test_pred)],
    'BestModelTrain': [accuracy_score(y_train, y_train_best_pred), f1_score(y_train, y_train_best_pred)],
    'BestModelTest': [accuracy_score(y_test, y_test_best_pred), f1_score(y_test, y_test_best_pred)]
})

print(metrics_comparison)

# 10) Train the Final Model on the entire dataset
final_model = best_pipeline.fit(X, y)

# 11) Save the Final Model
joblib.dump(final_model, 'final_XGB_model.pkl')

print(" ")
print("Final model saved as 'final_XGB_model.pkl'")


EVAL METRICS FOR BASE MODEL
Evaluation metrics for Train data:
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       400
           1       1.00      1.00      1.00       400

    accuracy                           1.00       800
   macro avg       1.00      1.00      1.00       800
weighted avg       1.00      1.00      1.00       800

Confusion Matrix:
[[400   0]
 [  0 400]]


Evaluation metrics for Test data:
Accuracy: 0.965
F1 Score: 0.9651741293532339
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       100
           1       0.96      0.97      0.97       100

    accuracy                           0.96       200
   macro avg       0.97      0.96      0.96       200
weighted avg       0.97      0.96      0.96       200

Confusion Matrix:
[[96  4]
 [ 3 97]]


Best Hyperparameters: {'classifier__learning_rat